In [1]:
import re
import wikipedia as wiki
from unidecode import unidecode
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint, TensorBoard
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

Using TensorFlow backend.


In [2]:
max_letters = 12
language_tags = {

                'en':['actor', 'alcohol', 'cheque', 'cancer', 'chocolate', 'debate', 'hobby', 'melon', 'propaganda',
                      'religion', 'violin', 'england', 'MediaWiki'],

                'si':["ශ්‍රී දළදා මාළිගාව",  "දියවඩන නිලමේ", "මහනුවර", "මාතලේ පරිපාලන දිස්ත්‍රික්කය", "ලංකා දිවයින","සංස්කෘතිය",
                      "න්‍යෂ්ටික අවි","තිරසාර සංවර්ධනය","කාන්තාව හා සංවර්ධනය","ආර්ථික සංවර්ධනය","මුල් ළමාවියේ සංවර්ධනය",
                     "ඵලදායී පාසල","රසායනික කර්මාන්ත","පළමුවන විමලධර්මසූරිය රජ","දළදා වහන්සේ","දියවඩන නිලමේ","මහනුවර","සමාජ වෙළඳ පොළ ආර්ථිකය",
                     "නාලන්දා විශ්වවිද්‍යාලය","CIA ගැන අභ්‍යන්තර සමාලෝචනය","CPU උපදෙස් මට්ටමේ සමාන්තරතාවය","ඔලිම්පික්","ශ්‍රී ලංකාවේ 14වන පාර්ලිමේන්තුව","හෘදය","උසස් පෙළ දේශපාලන විද්‍යාව",
                     "චීන ඉතිහාසය","ශ්‍රී ලංකාවේ 9වන පාර්ලිමේන්තුව","යුදෙවු ආගම","කොරියාවේ ඉතිහාසය","ජොසොන්","අක්නාටන්","ජන සන්නිවේදනය",
                     "ටයිටැනික් (නෞකාව)","සීගිරියේ සංස්කෘතික වටිනාකම්","කුමාරතුංග කැබිනට් මණ්ඩලය","චෝළ රාජවංශය","ධම්‌මහදයවිභඞ්‌ගො","බොරතෙල්","ඇඩොල්ෆ් හිට්ලර්",
                     "එක් දින ජාත්‍යන්තර ක්‍රිකට් වාර්තා ලැයිස්තුව","ශ්‍රී ලංකා යුද්ධ හමුදාව","ඉන්දු-යුරෝපීය භාෂා පවුල","ශ්‍රී ලංකාවේ 13වන පාර්ලිමේන්තුව",
                     "ගනේෂ","ශ්‍රී ලංකාවේ 12වන පාර්ලිමේන්තුව","ශ්‍රී ලංකාවේ 10වන පාර්ලිමේන්තුව","හන්ටිංටන්ගේ රෝගය","බියුටි ඇන්ඩ් ද බීස්ට් (2017 චිත්‍රපටය)"]

                 }

In [3]:
def generate_dictionary(tag, max_word_length):
    wiki.set_lang(tag)
    for topic in language_tags[tag]:
        # Scrap html content of webpage
        page = wiki.WikipediaPage(topic)
        #scrap plain text from page
        content = page.content
        #Unidecode the content
        content = unidecode(content)
        lst = process(content, max_word_length)
    print(tag+" word count: "+str(len(lst)))
    return lst

In [ ]:
def process(page_content, max_word_length):
    #get only word with regular expression
    words = re.sub(r'[^a-zA-Z ]', '', page_content)
    lower = words.lower()
    word_list = lower.split()
    short_words = []
    #check the word length with given max_word_length
    for word in word_list:
        if len(word) <= max_word_length:
            short_words.append(word)
    return short_words

In [ ]:
def convert_dic_to_vector(dic, max_word_length):
    new_list = []
    for word in dic:
        #output vector for one word
        vec = ''
        n = len(word)
        #convert charactor to one-hot-vector
        for i in range(n):
            #iterate each letter in word
            current_letter = word[i]
            #get the charactor position in a-z order
            ind = ord(current_letter)-97
            #Create one hot vector
            placeholder = (str(0)*ind) + str(1) + str(0)*(25-ind)
            #add char vector to word vector
            vec = vec + placeholder
        #create all waords as 12 length
        if n < max_word_length:
            excess = max_word_length-n
            vec = vec +str(0)*26*excess
        new_list.append(vec)
    return new_list

In [ ]:
def create_output_vector(tag_index, number_of_languages):
    #Create tags for outputs
    out = str(0)*tag_index + str(1) + str(0)*(number_of_languages-1-tag_index)
    return out

In [ ]:
#vector words list
word_data = []
#output vectors list
language_data = []
#word list
master_dic = []

#to keep the language count
count = 0

for tag in language_tags.keys():
    print('generating dictionary for ' + tag)
    #genarate dictionary for each language
    dic = generate_dictionary(tag, max_letters)
    #create main dictionary with all langs words
    for word in dic:
        master_dic.append(word)
    #vectorize the words in language dic
    vct = convert_dic_to_vector(dic, max_letters)
    #add vectorized word data in to word_data array
    for vector in vct:
        word_data.append(vector)
    #create output vectors
    output_vct = create_output_vector(count, len(language_tags))
    #add output vector data to language_data array
    for i in range(len(vct)):
        language_data.append(output_vct)
    count += 1

generating dictionary for en
en word count: 5901
generating dictionary for si


In [ ]:
arr = []

#create [word,word_vec,lang_vec] array and append in to arr
for i in range(len(word_data)):
    entry = []
    entry.append(master_dic[i])
    for digit in language_data[i]:
        entry.append(float(digit))
    for digit in word_data[i]:
        entry.append(float(digit))
    arr.append(entry)

#create dataframe with arr
arr = np.array(arr)
print(len(arr[1]))
np.save('arr.npy', arr)
df=pd.DataFrame(arr)
df.to_csv('data.csv')
print("Done")
print(type(arr))

In [ ]:
data = arr

inputs = data[:, 1+len(language_tags):]
labels = data[:, 1:1+len(language_tags)]

x_train, x_test, y_train, y_test = train_test_split(inputs, labels, test_size=0.15)

print(x_test.shape)
print(y_test.shape)
print(x_train.shape)
print(y_train.shape)

In [ ]:

network = Sequential()
network.add(Dense(200, input_dim=26*max_letters, activation='sigmoid'))
network.add(Dense(150, activation='sigmoid'))
network.add(Dense(100, activation='sigmoid'))
network.add(Dense(100, activation='sigmoid'))
network.add(Dense(len(language_tags), activation='softmax'))

network.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [ ]:
filepath = "weights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
tboard = TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
callbacks_list = [checkpoint, tboard]

network.fit(x_train, y_train, epochs=100, batch_size=1000, validation_data=(x_test, y_test), callbacks=callbacks_list)


In [ ]:
## Testing of the model
data = pd.read_csv('test_data_sets/en.txt', sep=" ", header=None)
data.columns = ['word']
for word in data['word']:
    netword.predict()